## app自动化测试
> 安装appium adb 模拟器

In [ ]:
import os
import re
import json
import threading
from time import sleep
from scapy.all import *
from appium.webdriver.common.appiumby import AppiumBy
from appium.webdriver import Remote

class Traffic:
    def __init__(self, name, count) -> None:
        self.filter = 'src 183.168.98.239 && tcp && !broadcast'
        self.name = name
        self.outdir = self.name+'.pcap'
        self.iface = 'Intel(R) Ethernet Connection (7) I219-LM'
        self.count = count
        self.times = time.time()

    def CallBack(self,packet):
        # print(packet)
        if packet.haslayer('HTTP'):
            http = packet.payload.payload.payload
            print(http.show())

    def timeout(self, x):
        now_time = time.time()
        if now_time - self.times > 30:
            return True

    def get(self):
        packet = sniff(prn=self.CallBack, iface=self.iface, filter=self.filter, count=self.count, timeout=30)
        wrpcap(self.outdir, packet)


class App_IUM:
    def __init__(self, appPackage, appActivity) -> None:
        self.appPackage = appPackage
        self.appActivity = appActivity
        #app args
        self.desired_cap={
            "platformName":"Android", 
            "platformVersion": "7.1.2",
            "deviceName":"HUAWEI",
            "automationName":"UiAutomator2",
            "appPackage":appPackage,
            "appActivity":appActivity,
        }
    def get_cap(self):
        return json.dumps(self.desired_cap)
    def active(self):
        os.system("adb shell monkey -p "+ self.appPackage +" --throttle 100 --pct-touch 100 100")
    def run(self, path, count):
        self.driver = Remote(command_executor='http://127.0.0.1:4723/wd/hub',desired_capabilities=self.desired_cap)
        traffic = Traffic(name=path, count=count)
        # thread pool
        threads = []
        # Syn calls
        thread_apk=threading.Thread(target=self.active)
        threads.append(thread_apk)
        thread_traffic=threading.Thread(target=traffic.get)
        threads.append(thread_traffic)
        thread_apk.start()
        thread_traffic.start()
        thread_apk.join()
        thread_traffic.join()

    def quit(self):
        self.driver.quit()
        # uninstall apk
        os.popen("adb uninstall "+self.appPackage).read()


class test(App_IUM):
    def __init__(self, appPath:str) -> None:
        """you can overwrite active() method, like add a element.click() 

        Args:
            appPath (str): give one app address
        """
        # install apk
        os.popen("adb install "+appPath).read()
        result = os.popen("aapt dump badging " + appPath).read()
        appPackage = re.compile("name='(.+?)' versionCode=").search(result).group(1)
        # print(appPackage)
        appActivity = re.compile("launchable-activity: name='(.+?)'  label=").search(result).group(1)
        super().__init__(appPackage, appActivity)


def get_name_5(l):
    l = l.replace("\\","/")
    return l[l.rfind("/")+1:l.rfind("/")+6]

def add_suffix(path):
    target_path = path
    for cls in os.listdir(target_path):
        if cls != "output":
            for name in os.listdir(os.path.join(target_path, cls)):
                if '.apk' not in os.path.join(target_path, cls, name):
                    os.rename(os.path.join(target_path, cls, name),os.path.join(target_path, cls, name)+".apk")

def uninstall_all_apk(appPath, num):
    for _, i in enumerate(os.listdir(appPath)):
        if num-2 < _ < num+1:
            try:
                path = os.path.join(appPath, i)
                os.popen("adb install "+path).read()
                result = os.popen("aapt dump badging " + path).read()
                appPackage = re.compile("name='(.+?)' versionCode=").search(result).group(1)
                os.popen("adb uninstall "+appPackage).read()
            except:
                pass


def autorun(data_path):
    add_suffix(data_path)
    for cls in os.listdir(data_path):
        if not os.path.exists(os.path.join("D:\data\drebin", 'output', cls)):
            os.mkdir(os.path.join("D:\data\drebin", 'output', cls))
        if cls != "output":
            for _,name in enumerate(os.listdir(os.path.join(data_path, cls))):
                # if _<130:
                #     continue
                name_5 = get_name_5(os.path.join(data_path,cls,name))
                print("\r{}[{}/{}]{}".format(cls,
                                        _+1,
                                        len(os.listdir(os.path.join(data_path, cls))),
                                        name_5),
                                        end="")
                try:
                    t = test(os.path.join(data_path,cls,name))
                    t.run(os.path.join(data_path, 'output', cls, name_5), 50)
                    t.quit()
                except:
                    pass

                uninstall_all_apk(os.path.join(data_path, cls), _+1)
autorun(data_path="D:\data\drebin")